In [1]:
import pickle
import numpy as np

In [2]:
ratings = pickle.load(open('items_10000_matrix.pickle', 'rb'), encoding='iso-8859-1')

In [3]:
type(ratings)

scipy.sparse.csr.csr_matrix

In [4]:
# ratings = ratings.toarray()

In [5]:
ratings.shape

(71869, 10000)

*ratings* -  np.array состоящий из нулей и единиц, где каждая строка соответсвует пользователю, а каждая колонка - книге.  
1 - на пересечении строки i колонки j значит, что пользователь i прочитал книгу j. 0 - пользователь i не читал книгу j.

# Part 1

### Task 1. Найти самую популярную книгу.

In [6]:
most_popular_book = ratings.sum(axis=0).argmax()
print('most popular book = {}'.format(most_popular_book))

most popular book = 20


### Task 2. Найти пользователя со второй по величине библиотекой.

In [7]:
second_biggest_library = np.asscalar(ratings.sum(axis=1).argsort(axis=0)[::-1][1])
print('user_id with the second biggest library = {}'.format(second_biggest_library))

user_id with the second biggest library = 69648


### Task 3. Найти среднее и медиану размера библиотеки.

In [8]:
mean = np.mean(ratings.sum(axis=1))
print('mean = {}'.format(mean))

median = int(np.median(ratings.sum(axis=1), axis=0))
print('median = {}'.format(median))

mean = 2.266693567463023
median = 2


### Task 4. Найти медиану размера библиоткеи у пользователей, которые прочитали книгу номер с id 100.

In [9]:
median_number = int(np.median(ratings[ratings.getcol(100).nonzero()[0]].sum(axis=1), axis=0))
print('Median number of books of users who read the book with id 100 = {}'.format(median_number))

Median number of books of users who read the book with id 100 = 3


# Part 2

### Task 1. Похожие книги.
1. Найди 20 наиболее похожих книг для книги с id = 100. В качестве меры похожести используй cosine similarity. Используй наиболее быстрый вариант подсчета.
  
$$cosine\ similarity = \frac{A\cdot B}{||A||\ ||B||} = \frac{\sum_{i=1}^{n} A_iB_i}{\sqrt{\sum_{i=1}^{n}A_i^2}\sqrt{\sum_{i=1}^{n}B_i^2}}$$  
  
В качестве ответа верни список кортежей, где каждый кортеж представляет собой пару (book_id, similarity).

In [10]:
%%time
from math import sqrt
COMP_BOOK = 100
aim = ratings.getcol(COMP_BOOK)

#users that have COMP_BOOK
users_COMP_BOOK = ratings[aim.nonzero()[0]]

#     b1 bComp | cos 
#u1    1   0   |  =0
#u2    0   0   |  =0
#u3    0   1   |  =0
#u4    1   1   |  >0 => \
#u5    1   1   |  >0 =>  > 3, count this in sums
#u6    1   1   |  >0 => /
#---------------
#total 4   4 => count this in total_sums

sums = users_COMP_BOOK.sum(axis=0)
total_sums = ratings[:,sums.nonzero()[1]].sum(axis=0).getA1()
# print(total_sums)

# count sum_i(Ai*Bi)
res_count = sums[:,sums.nonzero()[1]].getA1()
# print(res_count)

# book_ids
res_books = sums.nonzero()[1]
# print(res_books)
res_list = []

# count cos.sim.
for idx, val in enumerate(res_books):
    res_list.append((val, res_count[idx]/sqrt(total_sums[idx]*aim.size)))
    
# sort desc and take first 20 exclude COMP_BOOK (has cos.sim.=1.0) 
res_sample = sorted(res_list, key=lambda y: y[1])[::-1][1:21]
print("Answer: ")
print(res_sample)

Answer: 
[(5863, 0.2672612419124244), (5819, 0.2672612419124244), (5134, 0.1889822365046136), (5440, 0.1336306209562122), (3940, 0.1336306209562122), (2370, 0.1336306209562122), (3468, 0.1091089451179962), (5546, 0.08451542547285165), (1495, 0.06482037235521644), (214, 0.06482037235521644), (6, 0.048795003647426664), (528, 0.0445435403187374), (251, 0.03479445003196105), (836, 0.031943828249996996), (426, 0.03065696697424829), (312, 0.030261376633440123), (446, 0.025141822757713452), (73, 0.021262162778128115), (191, 0.016671298226202138), (76, 0.01566713146512808)]
Wall time: 110 ms


# Part 3

### Task 1. Матрица похожих книг (топ-20).

Создать матрицу похожих книг (матрица 10000 x 10000) использую метрику *cosine similarity*. В каждоый строке оставить только топ 20 самых похожих. Лучше написать функцию которая на вход получает матрицу рейтингов на выходе матрица похожих книг. Эту фунцкцию можно будет использовать в следующем задании.

In [12]:
from math import sqrt
from scipy.sparse import lil_matrix

def compute_sim_matrix(ratings):
    cols_count = ratings.shape[1]
    cossim_matrix = lil_matrix((cols_count, cols_count))
    
    total_sums = ratings.sum(axis=0).getA1()
    
    res_list = []
    for col_idx in range(cols_count):
        aim = ratings.getcol(col_idx)
        if aim.size == 0:
            continue;
        
        users_aim_book = ratings[aim.nonzero()[0]]

        sums = users_aim_book.sum(axis=0)
        res_count = sums[:,sums.nonzero()[1]].getA1()
        res_books = sums.nonzero()[1]
        
        res_list.clear()
        for idx, val in enumerate(res_books):
            res_list.append((val, res_count[idx]/sqrt(total_sums[val]*aim.size)))

        res_sample = sorted(res_list, key=lambda y: y[1])[::-1][:20]
        for (ibook_id, icossim) in res_sample:
            cossim_matrix[col_idx, ibook_id] = icossim
    
    return cossim_matrix

### Task 2. Оценить рекомендательную систему.

Оценить качество рекомендательной системы, основанной на матрице похожих книг, использую метрики **precision** и **recall**. Рекомендации должны получаться путем умножения матрицы похожих книг на вектор пользователя.
  
**precison** - кол-во книг которые попали пользователю в рекомендации и он их действительно добавил делить на кол-во рекомендованных книг.
  
**recall** - кол-во книг которые попали пользователю в рекомендации и он их действительно добавил делить на кол-во книг добавленных пользователем. 
  
Метрики считать на 20 самых лучших рекомендациях для каждого пользователя.

In [13]:
cossim_matrix = compute_sim_matrix(ratings)

In [14]:
%%time
import random as rnd
from scipy.sparse import lil_matrix
users_rec = ratings.dot(cossim_matrix).tolil()
users_bitmap = lil_matrix(users_rec.shape)

# set threshold, if user_rec[ibook] > threshold: ibook is recommend
t = 0.7
for i in range(users_rec.shape[0]):
    best_20 = sorted(users_rec.getrow(i).nonzero()[1], key=lambda x: users_rec[i, x])[::-1][:20]
    for j in best_20:
        if users_rec[i, j] > t:
            users_bitmap[i, j] = 1

TP = np.where((ratings + users_bitmap).data==2)[0].__len__()
print(TP)
            
precison = TP/users_bitmap.sum()
print('precison:', precison)

recall = TP/ratings.sum()
print('recall:', recall)

162895
precison: 0.9978009727173607
recall: 0.9999386145299408
Wall time: 1min 29s
